# Chapter 3: RAG (Retrieval-Augmented Generation) 실습

이 노트북은 다양한 RAG 기법과 전략을 실습합니다.

## 환경 설정

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("OpenAI API Key를 입력하세요: ")

## 1. 기본 RAG 구현

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 샘플 문서 준비
documents = [
    "LangChain은 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다.",
    "RAG는 검색 증강 생성의 약자로, 외부 지식을 활용하여 답변의 정확도를 높입니다.",
    "벡터 데이터베이스는 임베딩을 저장하고 유사도 검색을 수행합니다.",
    "프롬프트 엔지니어링은 LLM에게 효과적인 지시를 제공하는 기술입니다.",
    "체인은 여러 컴포넌트를 연결하여 복잡한 워크플로우를 구성합니다."
]

# 벡터 저장소 생성
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# RAG 프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "다음 컨텍스트를 바탕으로 질문에 답하세요:\n\n{context}"),
    ("human", "{question}")
])

# RAG 체인 구성
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# 질문하기
question = "RAG가 무엇인가요?"
answer = rag_chain.invoke(question)
print(f"질문: {question}")
print(f"답변: {answer}")

질문: RAG가 무엇인가요?
답변: RAG는 "검색 증강 생성"의 약자로, 외부 지식을 활용하여 답변의 정확도를 높이는 방법론입니다. 이 접근 방식은 주어진 질문에 대해 관련 정보를 검색하고, 이를 바탕으로 더 정확하고 풍부한 답변을 생성하는 데 도움을 줍니다. RAG는 주로 자연어 처리(NLP) 분야에서 사용되며, 정보 검색과 생성 모델을 결합하여 성능을 향상시키는 데 기여합니다.


## 2. Query Rewriting (질문 재작성)

In [3]:
# 질문 재작성 프롬프트
rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system", """사용자의 질문을 검색에 더 적합하도록 재작성하세요.
    모호한 표현을 명확하게 하고, 핵심 키워드를 포함시키세요."""),
    ("human", "{question}")
])

# 재작성 체인
rewrite_chain = rewrite_prompt | llm | StrOutputParser()

# 개선된 RAG 체인
improved_rag_chain = (
    {"question": rewrite_chain}
    | {"context": lambda x: retriever.invoke(x["question"]) | format_docs, 
       "question": lambda x: x["question"]}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# 테스트
original_question = "그거 뭐야?"
rewritten = rewrite_chain.invoke(original_question)
print(f"원본 질문: {original_question}")
print(f"재작성된 질문: {rewritten}")

# RAG로 답변
answer = rag_chain.invoke("LangChain이 뭐야?")
print(f"\n답변: {answer}")

원본 질문: 그거 뭐야?
재작성된 질문: "그거"가 무엇인지 구체적으로 알고 싶습니다. 어떤 주제나 사물에 대해 질문하시는 건가요? 예를 들어, 특정 제품, 개념, 사건 등에 대해 설명해 주시면 더 정확한 답변을 드릴 수 있습니다.

답변: LangChain은 언어 모델을 활용하여 애플리케이션을 개발할 수 있도록 돕는 프레임워크입니다. 이 프레임워크는 여러 컴포넌트를 연결하여 복잡한 워크플로우를 구성할 수 있게 해주며, 이를 통해 다양한 언어 처리 작업을 효율적으로 수행할 수 있습니다.


## 3. Multi-Query (다중 질문 생성)

In [4]:
from typing import List
from pydantic import BaseModel, Field

# 다중 질문 스키마
class MultiQueries(BaseModel):
    queries: List[str] = Field(description="생성된 질문 리스트")

# 다중 질문 생성 프롬프트
multi_query_prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 질문에 대해 다양한 관점에서 3개의 관련 질문을 생성하세요.
    각 질문은 원본 질문의 다른 측면을 다루어야 합니다."""),
    ("human", "{question}")
])

# 구조화된 출력으로 다중 질문 생성
multi_query_llm = llm.with_structured_output(MultiQueries)
multi_query_chain = multi_query_prompt | multi_query_llm

# 다중 질문으로 검색 수행
def multi_query_retrieval(question: str):
    # 다중 질문 생성
    multi_queries = multi_query_chain.invoke({"question": question})
    
    # 각 질문으로 검색
    all_docs = []
    for query in multi_queries.queries:
        docs = retriever.invoke(query)
        all_docs.extend(docs)
    
    # 중복 제거
    unique_docs = list({doc.page_content: doc for doc in all_docs}.values())
    return unique_docs

# 테스트
question = "LangChain으로 무엇을 할 수 있나요?"
queries = multi_query_chain.invoke({"question": question})
print(f"원본 질문: {question}\n")
print("생성된 질문들:")
for i, q in enumerate(queries.queries, 1):
    print(f"{i}. {q}")

# 다중 질문으로 검색
docs = multi_query_retrieval(question)
print(f"\n검색된 문서 수: {len(docs)}")
for doc in docs:
    print(f"- {doc.page_content[:50]}...")

원본 질문: LangChain으로 무엇을 할 수 있나요?

생성된 질문들:
1. LangChain을 사용하여 자연어 처리(NLP) 작업을 어떻게 개선할 수 있나요?
2. LangChain의 기능을 활용하여 데이터 분석에 어떤 이점을 제공할 수 있나요?
3. LangChain을 이용한 챗봇 개발에서의 장점은 무엇인가요?

검색된 문서 수: 3
- LangChain은 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다....
- 프롬프트 엔지니어링은 LLM에게 효과적인 지시를 제공하는 기술입니다....
- 체인은 여러 컴포넌트를 연결하여 복잡한 워크플로우를 구성합니다....


## 4. RAG Fusion

In [6]:
from typing import Dict

def reciprocal_rank_fusion(search_results: List[List], k: int = 60) -> List[str]:
    """Reciprocal Rank Fusion 알고리즘"""
    scores: Dict[str, float] = {}
    
    for result_list in search_results:
        for rank, doc in enumerate(result_list, 1):
            doc_id = doc.page_content
            if doc_id not in scores:
                scores[doc_id] = 0
            scores[doc_id] += 1 / (rank + k)
    
    # 점수순으로 정렬
    sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in sorted_docs]

# RAG Fusion 구현
def rag_fusion(question: str, num_queries: int = 3):
    # 1. 다중 질문 생성
    multi_queries = multi_query_chain.invoke({"question": question})
    
    # 2. 각 질문으로 검색
    all_results = []
    for query in multi_queries.queries[:num_queries]:
        docs = retriever.invoke(query)
        all_results.append(docs)
    
    # 3. RRF로 결과 융합
    fused_results = reciprocal_rank_fusion(all_results)
    
    return fused_results[:3]  # 상위 3개 반환

# 테스트
question = "벡터 데이터베이스의 역할은?"
fused_docs = rag_fusion(question)

print(f"질문: {question}\n")
print("RAG Fusion 결과:")
for i, doc in enumerate(fused_docs, 1):
    print(f"{i}. {doc}")

질문: 벡터 데이터베이스의 역할은?

RAG Fusion 결과:
1. 벡터 데이터베이스는 임베딩을 저장하고 유사도 검색을 수행합니다.
2. LangChain은 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다.
3. RAG는 검색 증강 생성의 약자로, 외부 지식을 활용하여 답변의 정확도를 높입니다.


## 5. HyDE (Hypothetical Document Embeddings)

In [8]:
# HyDE 프롬프트
hyde_prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 질문에 대한 가상의 답변을 작성하세요.
    이 답변은 실제 문서처럼 작성되어야 하며, 구체적이고 상세해야 합니다."""),
    ("human", "{question}")
])

# HyDE 체인
hyde_chain = hyde_prompt | llm | StrOutputParser()

def hyde_retrieval(question: str):
    # 1. 가상 문서 생성
    hypothetical_doc = hyde_chain.invoke({"question": question})
    print(f"생성된 가상 문서:\n{hypothetical_doc}\n")
    
    # 2. 가상 문서로 검색
    docs = vectorstore.similarity_search(hypothetical_doc, k=3)
    return docs

# 테스트
question = "프롬프트 엔지니어링의 중요성은?"
print(f"질문: {question}\n")

hyde_docs = hyde_retrieval(question)
print("검색된 실제 문서:")
for i, doc in enumerate(hyde_docs, 1):
    print(f"{i}. {doc.page_content}")

질문: 프롬프트 엔지니어링의 중요성은?

생성된 가상 문서:
프롬프트 엔지니어링(Prompt Engineering)은 인공지능 모델, 특히 자연어 처리(NLP) 모델과의 상호작용에서 매우 중요한 역할을 합니다. 이는 사용자가 원하는 결과를 얻기 위해 모델에 제공하는 입력(프롬프트)을 설계하고 최적화하는 과정을 의미합니다. 다음은 프롬프트 엔지니어링의 중요성을 구체적으로 설명하는 몇 가지 이유입니다.

### 1. 모델 성능 극대화
프롬프트 엔지니어링은 모델의 성능을 극대화하는 데 필수적입니다. 적절한 프롬프트를 사용하면 모델이 더 정확하고 관련성 높은 응답을 생성할 수 있습니다. 예를 들어, 특정 질문에 대한 명확한 지침을 제공함으로써 모델이 더 나은 이해를 바탕으로 답변을 생성하도록 유도할 수 있습니다.

### 2. 사용자 경험 향상
사용자가 인공지능 모델과 상호작용할 때, 프롬프트의 품질은 최종 결과물의 품질에 직접적인 영향을 미칩니다. 잘 설계된 프롬프트는 사용자가 원하는 정보를 더 쉽게 얻을 수 있도록 도와주며, 이는 전반적인 사용자 경험을 향상시킵니다. 예를 들어, 명확하고 구체적인 질문을 통해 사용자는 더 유용한 정보를 얻을 수 있습니다.

### 3. 다양한 응용 가능성
프롬프트 엔지니어링은 다양한 분야에서 응용될 수 있습니다. 예를 들어, 고객 지원, 콘텐츠 생성, 데이터 분석 등 여러 산업에서 프롬프트를 최적화함으로써 특정 요구 사항에 맞는 결과를 도출할 수 있습니다. 이는 기업이 인공지능 기술을 활용하여 경쟁력을 높이는 데 기여합니다.

### 4. 모델의 한계 이해
프롬프트 엔지니어링을 통해 사용자는 모델의 한계를 이해하고 이를 극복할 수 있는 방법을 모색할 수 있습니다. 특정 프롬프트가 예상치 못한 결과를 초래할 경우, 이를 분석하고 수정함으로써 모델의 이해도를 높이고, 더 나은 결과를 얻을 수 있는 방법을 찾을 수 있습니다.

### 5. 지속적인 개선
프롬프트 엔지니어링은 반복적인 과정입니다. 사용자는 모델의 응답을 평가하고, 이를 

## 6. 의도 기반 라우팅

In [9]:
from enum import Enum

class QueryType(str, Enum):
    DEFINITION = "definition"
    COMPARISON = "comparison"
    HOWTO = "howto"
    GENERAL = "general"

class QueryClassification(BaseModel):
    query_type: QueryType = Field(description="질문의 유형")
    reasoning: str = Field(description="분류 이유")

# 질문 분류 프롬프트
classify_prompt = ChatPromptTemplate.from_messages([
    ("system", """질문을 다음 카테고리 중 하나로 분류하세요:
    - definition: 정의나 개념을 묻는 질문
    - comparison: 비교를 요구하는 질문
    - howto: 방법이나 절차를 묻는 질문
    - general: 일반적인 질문"""),
    ("human", "{question}")
])

# 분류기
classifier = llm.with_structured_output(QueryClassification)
classify_chain = classify_prompt | classifier

# 각 유형별 프롬프트
prompts = {
    QueryType.DEFINITION: "정의: {context}를 바탕으로 '{question}'을(를) 명확하게 정의하세요.",
    QueryType.COMPARISON: "비교: {context}를 바탕으로 '{question}'에 대해 비교 설명하세요.",
    QueryType.HOWTO: "방법: {context}를 바탕으로 '{question}'을(를) 수행하는 단계를 설명하세요.",
    QueryType.GENERAL: "일반: {context}를 바탕으로 '{question}'에 답하세요."
}

def routed_rag(question: str):
    # 1. 질문 분류
    classification = classify_chain.invoke({"question": question})
    print(f"질문 유형: {classification.query_type}")
    print(f"이유: {classification.reasoning}\n")
    
    # 2. 문서 검색
    docs = retriever.invoke(question)
    context = format_docs(docs)
    
    # 3. 유형별 프롬프트 선택
    prompt_template = prompts[classification.query_type]
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 도움이 되는 AI 어시스턴트입니다."),
        ("human", prompt_template)
    ])
    
    # 4. 답변 생성
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"context": context, "question": question})
    return answer

# 테스트
test_questions = [
    "LangChain이 뭔가요?",
    "RAG와 일반 LLM의 차이점은?",
    "벡터 데이터베이스를 어떻게 사용하나요?"
]

for q in test_questions:
    print(f"\n질문: {q}")
    print("-" * 50)
    answer = routed_rag(q)
    print(f"답변: {answer}\n")


질문: LangChain이 뭔가요?
--------------------------------------------------
질문 유형: QueryType.DEFINITION
이유: 질문은 LangChain의 정의나 개념을 묻고 있으므로 'definition' 카테고리에 해당합니다.

답변: LangChain은 언어 모델을 활용하여 애플리케이션을 개발할 수 있도록 지원하는 프레임워크입니다. 이 프레임워크는 다양한 컴포넌트를 연결하여 복잡한 워크플로우를 구성할 수 있게 하여, 개발자들이 자연어 처리(NLP) 기능을 쉽게 통합하고 활용할 수 있도록 돕습니다. LangChain을 사용하면 언어 모델의 기능을 조합하여 다양한 애플리케이션을 효율적으로 구축할 수 있습니다.


질문: RAG와 일반 LLM의 차이점은?
--------------------------------------------------
질문 유형: QueryType.COMPARISON
이유: 질문은 RAG(리트리벌-어그멘테이션-제너레이션)와 일반 LLM(대형 언어 모델)의 차이점을 비교하고 있으므로, 비교를 요구하는 질문으로 분류됩니다.

답변: RAG(검색 증강 생성)와 일반 LLM(대형 언어 모델)의 차이점은 주로 정보 처리 방식과 응답의 정확성에 있습니다. 아래에서 두 가지 접근 방식을 비교해 보겠습니다.

### 1. 정보 출처
- **RAG**: RAG는 외부 데이터베이스나 검색 엔진을 활용하여 관련 정보를 검색한 후, 이를 바탕으로 응답을 생성합니다. 이 과정에서 최신 정보나 특정 도메인에 대한 전문 지식을 포함할 수 있어, 보다 정확하고 신뢰할 수 있는 답변을 제공합니다.
- **일반 LLM**: 일반 LLM은 훈련 데이터에 기반하여 응답을 생성합니다. 이 데이터는 고정되어 있으며, 모델이 훈련된 시점 이후의 정보나 특정 세부사항에 대한 접근이 제한적입니다. 따라서 최신 정보나 특정 질문에 대한 정확한 답변을 제공하기 어려울 수 있습니다.

### 2. 응답의 정확성
- **RAG*

## 7. Self-Query Retriever

In [10]:
from langchain.schema import Document
from typing import Optional

# 메타데이터가 있는 문서 생성
docs_with_metadata = [
    Document(
        page_content="Python은 1991년에 귀도 반 로섬이 개발한 프로그래밍 언어입니다.",
        metadata={"year": 1991, "category": "programming", "language": "Python"}
    ),
    Document(
        page_content="JavaScript는 1995년에 브렌던 아이크가 개발한 스크립트 언어입니다.",
        metadata={"year": 1995, "category": "programming", "language": "JavaScript"}
    ),
    Document(
        page_content="머신러닝은 2010년대에 딥러닝과 함께 급속히 발전했습니다.",
        metadata={"year": 2010, "category": "ai", "topic": "machine learning"}
    ),
    Document(
        page_content="트랜스포머 모델은 2017년에 구글이 발표한 혁신적인 아키텍처입니다.",
        metadata={"year": 2017, "category": "ai", "topic": "transformer"}
    ),
]

# 벡터 저장소 생성
metadata_vectorstore = FAISS.from_documents(docs_with_metadata, embeddings)

# Self-Query 파싱
class SelfQuery(BaseModel):
    query: str = Field(description="검색할 내용")
    category: Optional[str] = Field(default=None, description="카테고리 필터")
    year_min: Optional[int] = Field(default=None, description="최소 연도")
    year_max: Optional[int] = Field(default=None, description="최대 연도")

# Self-Query 프롬프트
self_query_prompt = ChatPromptTemplate.from_messages([
    ("system", """사용자의 질문을 분석하여 검색 쿼리와 필터를 추출하세요.
    카테고리: programming, ai
    연도 범위도 추출할 수 있습니다."""),
    ("human", "{question}")
])

self_query_llm = llm.with_structured_output(SelfQuery)
self_query_chain = self_query_prompt | self_query_llm

def self_query_search(question: str):
    # 쿼리 파싱
    parsed = self_query_chain.invoke({"question": question})
    print(f"파싱된 쿼리: {parsed}\n")
    
    # 검색 수행
    results = metadata_vectorstore.similarity_search(parsed.query, k=5)
    
    # 메타데이터 필터링
    filtered = []
    for doc in results:
        if parsed.category and doc.metadata.get("category") != parsed.category:
            continue
        if parsed.year_min and doc.metadata.get("year", 0) < parsed.year_min:
            continue
        if parsed.year_max and doc.metadata.get("year", float('inf')) > parsed.year_max:
            continue
        filtered.append(doc)
    
    return filtered

# 테스트
test_queries = [
    "2000년 이전에 개발된 프로그래밍 언어",
    "AI 분야의 최신 기술",
    "프로그래밍 언어의 역사"
]

for q in test_queries:
    print(f"질문: {q}")
    results = self_query_search(q)
    print("검색 결과:")
    for doc in results:
        print(f"- {doc.page_content}")
        print(f"  메타데이터: {doc.metadata}")
    print("\n" + "="*60 + "\n")

질문: 2000년 이전에 개발된 프로그래밍 언어
파싱된 쿼리: query='프로그래밍 언어' category='programming' year_min=None year_max=2000

검색 결과:
- Python은 1991년에 귀도 반 로섬이 개발한 프로그래밍 언어입니다.
  메타데이터: {'year': 1991, 'category': 'programming', 'language': 'Python'}
- JavaScript는 1995년에 브렌던 아이크가 개발한 스크립트 언어입니다.
  메타데이터: {'year': 1995, 'category': 'programming', 'language': 'JavaScript'}


질문: AI 분야의 최신 기술
파싱된 쿼리: query='최신 AI 기술' category='ai' year_min=2023 year_max=2023

검색 결과:


질문: 프로그래밍 언어의 역사
파싱된 쿼리: query='프로그래밍 언어의 역사' category='programming' year_min=None year_max=None

검색 결과:
- Python은 1991년에 귀도 반 로섬이 개발한 프로그래밍 언어입니다.
  메타데이터: {'year': 1991, 'category': 'programming', 'language': 'Python'}
- JavaScript는 1995년에 브렌던 아이크가 개발한 스크립트 언어입니다.
  메타데이터: {'year': 1995, 'category': 'programming', 'language': 'JavaScript'}




## 8. SQL 예제

In [11]:
import sqlite3
import pandas as pd

# SQLite 데이터베이스 생성
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 테이블 생성
cursor.execute('''
    CREATE TABLE products (
        id INTEGER PRIMARY KEY,
        name TEXT,
        category TEXT,
        price REAL,
        stock INTEGER
    )
''')

# 샘플 데이터 삽입
products = [
    (1, 'MacBook Pro', 'Laptop', 2499.99, 10),
    (2, 'iPhone 15', 'Phone', 999.99, 25),
    (3, 'AirPods Pro', 'Audio', 249.99, 50),
    (4, 'iPad Air', 'Tablet', 599.99, 15),
    (5, 'Apple Watch', 'Wearable', 399.99, 30)
]

cursor.executemany('INSERT INTO products VALUES (?, ?, ?, ?, ?)', products)
conn.commit()

# SQL 생성 프롬프트
sql_prompt = ChatPromptTemplate.from_messages([
    ("system", """다음 테이블 스키마를 참고하여 자연어 질문을 SQL 쿼리로 변환하세요:
    
    products 테이블:
    - id: INTEGER (PRIMARY KEY)
    - name: TEXT
    - category: TEXT
    - price: REAL
    - stock: INTEGER
    
    SQL 쿼리만 반환하고 다른 설명은 포함하지 마세요."""),
    ("human", "{question}")
])

sql_chain = sql_prompt | llm | StrOutputParser()

def execute_sql_query(question: str):
    # SQL 생성
    sql_query = sql_chain.invoke({"question": question})
    sql_query = sql_query.replace("```sql", "").replace("```", "").strip()
    print(f"생성된 SQL: {sql_query}\n")
    
    try:
        # SQL 실행
        result = pd.read_sql_query(sql_query, conn)
        return result
    except Exception as e:
        return f"에러: {e}"

# 테스트
questions = [
    "가장 비싼 제품은?",
    "재고가 20개 이상인 제품들",
    "카테고리별 평균 가격"
]

for q in questions:
    print(f"질문: {q}")
    result = execute_sql_query(q)
    print("결과:")
    print(result)
    print("\n" + "="*60 + "\n")

질문: 가장 비싼 제품은?
생성된 SQL: SELECT * FROM products ORDER BY price DESC LIMIT 1;

결과:
   id         name category    price  stock
0   1  MacBook Pro   Laptop  2499.99     10


질문: 재고가 20개 이상인 제품들
생성된 SQL: SELECT * FROM products WHERE stock >= 20;

결과:
   id         name  category   price  stock
0   2    iPhone 15     Phone  999.99     25
1   3  AirPods Pro     Audio  249.99     50
2   5  Apple Watch  Wearable  399.99     30


질문: 카테고리별 평균 가격
생성된 SQL: SELECT category, AVG(price) AS average_price
FROM products
GROUP BY category;

결과:
   category  average_price
0     Audio         249.99
1    Laptop        2499.99
2     Phone         999.99
3    Tablet         599.99
4  Wearable         399.99




## 실습 과제

1. 여러 RAG 기법을 조합한 하이브리드 시스템 구현
2. 평가 메트릭을 추가하여 각 기법의 성능 비교
3. 실제 문서를 사용한 도메인 특화 RAG 시스템 구축

In [ ]:
# 여기에 실습 코드를 작성하세요
